In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
from imutils.video import VideoStream
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
# import time
# import cv2
# import os
import dlib
from scipy.spatial import distance as dist
x = 0

pygame 2.0.1 (SDL 2.0.14, Python 3.7.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Eyes close and open
model1 = load_model('trained_model_24_2.h5')
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 18, 18, 64)        0

In [3]:
#Liveness detector

print("loading face detector")
protoPath =  'deploy.prototxt'

#Loading the caffe model 
modelPath = 'res10_300x300_ssd_iter_140000.caffemodel'

#reading data from the model.
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# loading the liveness detecting module that was trained in the training python script
print("loading the liveness detector")
model2 = load_model('liveness.model')
le = pickle.loads(open('le.pickle', "rb").read())

loading face detector
loading the liveness detector


D:\Python\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.2 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
#determining the facial points that are plotted by dlib
FULL_POINTS = list(range(0, 68))  
FACE_POINTS = list(range(17, 68))  
   
JAWLINE_POINTS = list(range(0, 17))  
RIGHT_EYEBROW_POINTS = list(range(17, 22))  
LEFT_EYEBROW_POINTS = list(range(22, 27))  
NOSE_POINTS = list(range(27, 36))  
RIGHT_EYE_POINTS = list(range(36, 42))  
LEFT_EYE_POINTS = list(range(42, 48))  
MOUTH_OUTLINE_POINTS = list(range(48, 61))  
MOUTH_INNER_POINTS = list(range(61, 68))  
   
EYE_AR_THRESH = 0.30 
EYE_AR_CONSEC_FRAMES = 2  

#initializing the parameters
COUNTER_LEFT = 0  
TOTAL_LEFT = 0  
   
COUNTER_RIGHT = 0  
TOTAL_RIGHT = 0 

In [5]:
#defining a function for calculating ear and then comparing with the confidence parametrs

def eye_aspect_ratio(eye):
    
    A = dist.euclidean(eye[1], eye[5])  
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])  
    ear = (A + B) / (2.0 * C)  
    return ear 

In [6]:
#loading the predictor for predicting
detector = dlib.get_frontal_face_detector()  

#accessing the shape predictor
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [12]:
mixer.init()
sound = mixer.Sound('TF050_2.wav')

face = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')



lbl=['Close','Open']


# path = os.getcwd()
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]
alarm_threshold = 10
# video_capture = cv2.VideoCapture(0)  
fake_count = 0

In [17]:
cap = cv2.VideoCapture(0)

In [18]:
while(True):
    ret, frame = cap.read()
    fram1 = frame.copy()
    
    if ret:
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
        rects = detector(gray, 0)
        frame = imutils.resize(frame, width=600)
        height,width = frame.shape[:2]
        for rect in rects:
            
            x = rect.left()  
            y = rect.top()  
            x1 = rect.right()  
            y1 = rect.bottom()
            landmarks = np.matrix([[p.x, p.y] for p in predictor(frame, rect).parts()])  
            left_eye = landmarks[LEFT_EYE_POINTS]  
            right_eye = landmarks[RIGHT_EYE_POINTS]  
            left_eye_hull = cv2.convexHull(left_eye)  
            right_eye_hull = cv2.convexHull(right_eye)  
            ear_left = eye_aspect_ratio(left_eye)  
            ear_right = eye_aspect_ratio(right_eye)
            
#             faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
#             cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
#             for (x,y,w,h) in faces:
#                 cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
            
#             cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc)
            
            
            #calculating blink wheneer the ear value drops down below the threshold
            frame = imutils.resize(frame, width=600)
            
            if ear_left < EYE_AR_THRESH:
                
                COUNTER_LEFT += 1
            
            else:
                
                
                if COUNTER_LEFT >= EYE_AR_CONSEC_FRAMES:
                    
                    
                    TOTAL_LEFT += 1  
                    print("Left eye winked") 
                
                    COUNTER_LEFT = 0
            if ear_right < EYE_AR_THRESH:  
                
                
                COUNTER_RIGHT += 1  

            else:
                
                if COUNTER_RIGHT >= EYE_AR_CONSEC_FRAMES: 
                    
                    
                    TOTAL_RIGHT += 1  
                    print("Right eye winked")  
                    COUNTER_RIGHT = 0


            x = TOTAL_LEFT + TOTAL_RIGHT
            
            
    (h, w) = frame.shape[:2]
    temp = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
		(300, 300), (104.0, 177.0, 123.0))
    net.setInput(temp)
    detections = net.forward()
    label = 0
    p=0
    sX=0
    sY=0
    eX = 0
    eY = 0
    for i in range(0, detections.shape[2]):


        confidence = detections[0, 0, i, 2]

          #staisfying the union need of veryfying through ROI and blink detection.  
        if confidence > 0.5 and x>10:



            #detect a bounding box
        #take dimensions
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        #get the dimensions
            (startX, startY, endX, endY) = box.astype("int")


            sX=startX = max(0, startX)
            sY=startY = max(0, startY)
            eX=endX = min(w, endX)
            eY=endY = min(h, endY)

        # extract the face ROI and then preproces it in the exact
        # same manner as our training data
            face = frame[startY:endY, startX:endX]
            face = cv2.resize(face, (32, 32))
            face = face.astype("float") / 255.0
            face = img_to_array(face)
            face = np.expand_dims(face, axis=0)

        #pass the model to determine the liveness
            preds = model2.predict(face)[0]
            j = np.argmax(preds)
            label_1 = le.classes_[1-j] 
            
            p = preds[j]
            label = label_1.decode('UTF-8')
            
            
    if(label == 'fake'):
        fake_count += 1
        if(fake_count >= 10):
            try:
                sound.play()
            except:
                pass
    else:
        fake_count = 0
    
    if(type(label) == int):
        label = str(label)
    label += (" " + str(p))
    cv2.putText(frame,label,(250,height-10), font, 1,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(frame, label, (sX, sY - 10),
	cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.rectangle(frame, (sX, sY), (eX, eY),
				  (0, 0, 255), 2)

    left_eye_1 = leye.detectMultiScale(gray)
    right_eye_1 =  reye.detectMultiScale(gray)
    r_eye,l_eye = 0,0

    for (a,b,c,d) in right_eye_1:
        r_eye=frame[b:b+d,a:a+c]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))                       #
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)                 #
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = model1.predict_classes(r_eye)
        if(rpred[0]==1):
            lbl='Open' 
        if(rpred[0]==0):
            lbl='Closed'
        break

    for (a,b,c,d) in left_eye_1:
        l_eye=frame[b:b+d,a:a+c]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))                     #
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)               #
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = model1.predict_classes(l_eye)
        if(lpred[0]==1):
            lbl='Open'   
        if(lpred[0]==0):
            lbl='Closed'
        break

    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-10), font, 1,(255,255,255),1,cv2.LINE_AA)
    elif(rpred[0]==1 or lpred[0]==1):
        score = 0
        cv2.putText(frame,"Open",(10,height-10), font, 1,(255,255,255),1,cv2.LINE_AA)
    else:
        score=score-1
        #score = 0
        cv2.putText(frame,"Open",(10,height-10), font, 1,(255,255,255),1,cv2.LINE_AA)

    if(score<0):
        score=0   
    cv2.putText(frame,'Score:'+str(score),(100,height-10), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>alarm_threshold):
        try:
            sound.play()

        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2

            
            
            
            
            
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

C:\Users\ARYAN\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Right eye winked
Left eye winked
Right eye winked
Left eye winked
Right eye winked
Right eye winked
Right eye winked
Left eye winked
Right eye winked
Right eye winked
Right eye winked
Left eye winked
Right eye winked
Right eye winked
Right eye winked
Right eye winked
Right eye winked
Right eye winked
Left eye winked
Left eye winked
Right eye winked
Left eye winked
Left eye winked
Right eye winked
Left eye winked
Right eye winked
Left eye winked
Right eye winked
Left eye winked
Right eye winked
Left eye winked
Left eye winked
Right eye winked
Right eye winked
Left eye winked
Left eye winked
Right eye winked
Left eye winked
Left eye winked
Left eye winked
Right eye winked
Right eye winked
Left eye winked
Right eye winked
Left eye winked
Right eye winked


In [13]:
cap.release()
cv2.destroyAllWindows()